In [283]:
%reload_ext autoreload
%autoreload 2

In [286]:
import pandas as pd
from pathlib import Path

from mbp.webscraping import activate_web_driver, get_team_page
from mbp.data import download_team_data, get_team_games_for_year
from mbp.paths import SEASONS_DIR, RAW_DATA_DIR
from mbp.models import TeamYear, TeamGame

In [144]:
team_name = "Arizona"
raw_year = 2022

In [145]:
driver = activate_web_driver("firefox", True)

# Read team names
df = pd.read_csv(f"{RAW_DATA_DIR}/mbb_team_names_to_number.csv", index_col=0)

In [147]:
games_df = get_team_games_for_year(driver, df, team_name, raw_year)
games_df.head()

Opp has a # in it: #10 Stanford
@Las Vegas, NV (2023 MBB Pac-12 Championship)
Opp has a # in it: #6 Arizona St.
@Las Vegas, NV (2023 MBB Pac-12 Championship)
Opp has a # in it: #1 UCLA
@Las Vegas, NV (2023 MBB Pac-12 Championship)
Opp has a # in it: #15 Princeton
@Sacramento, CA (2023 NCAA Division I Men's Basketball Championship)


,raw_datetime,opponent,result,attendance,datetime,home
0,11/11/2022,Southern U.,W 95-78,"13,485",2022-11-11,1.0
1,11/17/2022,Utah Tech,W 104-77,"12,752",2022-11-17,1.0
2,11/21/2022,Cincinnati,W 101-93,"2,400",2022-11-21,0.0
3,11/22/2022,San Diego St.,W 87-70,"2,400",2022-11-22,0.0
4,11/23/2022,Creighton,W 81-79,"2,400",2022-11-23,0.0


In [148]:
# Download Arizona data
download_team_data(team_name, raw_year, True)

Opp has a # in it: #10 Stanford
@Las Vegas, NV (2023 MBB Pac-12 Championship)
Opp has a # in it: #6 Arizona St.
@Las Vegas, NV (2023 MBB Pac-12 Championship)
Opp has a # in it: #1 UCLA
@Las Vegas, NV (2023 MBB Pac-12 Championship)
Opp has a # in it: #15 Princeton
@Sacramento, CA (2023 NCAA Division I Men's Basketball Championship)


In [149]:
# Get their games for the year
year_dir = Path(SEASONS_DIR / str(raw_year))
team_dir = year_dir / team_name
games_df = pd.read_csv(team_dir / "games.csv")

for opp in games_df['opponent']:
    opponent_games = year_dir / opp / "games.csv"
    if not opponent_games.exists():
        print(opp)
        download_team_data(opp, raw_year)

Princeton


In [273]:
from mbp.models import TeamYear, TeamGame

In [330]:
az = TeamYear("Arizona", 2022)
az.download_updated_data(True)

games = az.get_games()

/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:234: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '95' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "team_score"] = score_parts[0]
/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:235: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '78' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "opp_score"] = score_parts[1]


In [331]:
from datetime import datetime, timedelta, date
today = date.today()
today = today + timedelta(days=-290)
print(today.strftime("%Y-%m-%d"))

df2 = games.copy()
df2['datetime'] = pd.to_datetime(df2['datetime'])
df2['difference'] = (df2['datetime'].dt.date - today)
# next_row = df.loc[df['difference'] == df['difference'].min()]
# print(df2.loc[df2['TimeDifference'].min()])
future_dates = df2[df2['difference'] > timedelta(days=0)]
next_future_date = future_dates.loc[future_dates['difference'].idxmin()]
next_future_date

2022-12-31


opponent               Washington
result                      70-67
attendance                 13,562
datetime      2023-01-05 00:00:00
win                           1.0
team_score                     70
opp_score                      67
home                          1.0
difference        5 days, 0:00:00
Name: 13, dtype: object

In [373]:
from mbp.models import get_next_game

today = date.today()
last_year_date = today + timedelta(days=-170)

az.get_next_opponent_or_last(last_year_date)
game = get_next_game(az, last_year_date)
game.date
# game.get_players()

opponent                Princeton
result                      55-59
attendance                 14,896
datetime      2023-03-16 00:00:00
win                           0.0
team_score                     55
opp_score                      59
home                          0.0
difference      -45 days, 0:00:00
Name: 33, dtype: object


Timestamp('2023-03-16 00:00:00')

AttributeError: 'Timestamp' object has no attribute 'dt'